## SQL GENERATION llm->GPT3.5turbo\
### 필요모듈 import

In [1]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

from sqlalchemy import insert
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index import Document, ListIndex
from llama_index import SQLDatabase, ServiceContext
from llama_index.llms import ChatMessage, OpenAI
from typing import List
import ast
import openai
from IPython.display import display, HTML

## sample data  

In [2]:
rows = [
    # iPhone13 Reviews
    {"id": 1, "category": "Phone", "product_name": "Iphone13", "review": "Amazing battery life and camera quality. Best iPhone yet."},
    {"id": 2, "category": "Phone", "product_name": "Iphone13", "review": "Stunning design and performance. Apple has done it again."},
    {"id": 3, "category": "Phone", "product_name": "Iphone13", "review": "The display is just incredible. Love the A15 chip's speed."},
    {"id": 4, "category": "Phone", "product_name": "Iphone13", "review": "Superb user experience with the new iOS. Seamless and smooth."},
    {"id": 5, "category": "Phone", "product_name": "Iphone13", "review": "5G capabilities are outstanding. Internet browsing is lightning fast."},
    {"id": 6, "category": "Phone", "product_name": "Iphone13", "review": "The build quality is top-notch. Feels premium in hand."},
    {"id": 7, "category": "Phone", "product_name": "Iphone13", "review": "Love the enhanced camera system. Pictures are more vivid."},
    {"id": 8, "category": "Phone", "product_name": "Iphone13", "review": "Face ID is faster and more responsive."},
    {"id": 9, "category": "Phone", "product_name": "Iphone13", "review": "Storage options are great. Worth the investment."},
    {"id": 10, "category": "Phone", "product_name": "Iphone13", "review": "Night mode photos are simply outstanding."},
    {"id": 11, "category": "Phone", "product_name": "Iphone13", "review": "The ceramic shield front cover is a nice touch."},
    {"id": 12, "category": "Phone", "product_name": "Iphone13", "review": "Battery life lasts all day even with heavy use."},
    {"id": 13, "category": "Phone", "product_name": "Iphone13", "review": "Video recording quality is cinema-grade."},
    {"id": 14, "category": "Phone", "product_name": "Iphone13", "review": "Gaming experience is smooth with no lags."},
    {"id": 15, "category": "Phone", "product_name": "Iphone13", "review": "Dual eSIM support is a game changer for travelers."},
    {"id": 16, "category": "Phone", "product_name": "Iphone13", "review": "Stereo speakers produce clear and loud audio."},
    {"id": 17, "category": "Phone", "product_name": "Iphone13", "review": "MagSafe accessories add to its versatility."},
    {"id": 18, "category": "Phone", "product_name": "Iphone13", "review": "Water and dust resistance is reliable."},
    {"id": 19, "category": "Phone", "product_name": "Iphone13", "review": "Graphics and performance are top-tier."},
    {"id": 20, "category": "Phone", "product_name": "Iphone13", "review": "All-around, the best smartphone on the market."},

    # SamsungTV Reviews
    {"id": 21, "category": "TV", "product_name": "SamsungTV", "review": "Impressive picture clarity and vibrant colors. A top-notch TV."},
    {"id": 22, "category": "TV", "product_name": "SamsungTV", "review": "Love the smart features and the remote. Simplifies everything."},
    {"id": 23, "category": "TV", "product_name": "SamsungTV", "review": "Sound quality could be better. Picture is stunning though."},
    {"id": 24, "category": "TV", "product_name": "SamsungTV", "review": "Connectivity options are plenty and easy to set up."},
    {"id": 25, "category": "TV", "product_name": "SamsungTV", "review": "Sleek design that complements the living room decor."},
    {"id": 26, "category": "TV", "product_name": "SamsungTV", "review": "The built-in apps and interface are user-friendly."},
    {"id": 27, "category": "TV", "product_name": "SamsungTV", "review": "Ambient mode is a nice touch for when not watching."},
    {"id": 28, "category": "TV", "product_name": "SamsungTV", "review": "Gaming mode offers a fantastic experience with low latency."},
    {"id": 29, "category": "TV", "product_name": "SamsungTV", "review": "HDR content looks spectacular."},
    {"id": 30, "category": "TV", "product_name": "SamsungTV", "review": "Remote control with voice command is super convenient."},
    {"id": 31, "category": "TV", "product_name": "SamsungTV", "review": "Crisp and clear audio, though an external system enhances it."},
    {"id": 32, "category": "TV", "product_name": "SamsungTV", "review": "Wall mounting was straightforward and secure."},
    {"id": 33, "category": "TV", "product_name": "SamsungTV", "review": "Streaming apps load quickly and play smoothly."},
    {"id": 34, "category": "TV", "product_name": "SamsungTV", "review": "Regular software updates keep the TV fresh."},
    {"id": 35, "category": "TV", "product_name": "SamsungTV", "review": "Multiple HDMI ports allow for various device connections."},
    {"id": 36, "category": "TV", "product_name": "SamsungTV", "review": "Power consumption is efficient."},
    {"id": 37, "category": "TV", "product_name": "SamsungTV", "review": "Tizen OS is intuitive and bug-free."},
    {"id": 38, "category": "TV", "product_name": "SamsungTV", "review": "The contrast ratio and deep blacks are impressive."},
    {"id": 39, "category": "TV", "product_name": "SamsungTV", "review": "No motion blur during fast-paced scenes."},
    {"id": 40, "category": "TV", "product_name": "SamsungTV", "review": "Overall, a solid investment for quality viewing."},

    # Ergonomic Chair Reviews
    {"id": 41, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Feels really comfortable even after long hours."},
    {"id": 42, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Assembly was a bit tough, but the comfort is unmatched."},
    {"id": 43, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Good support for back but wish it had more adjustability."},
    {"id": 44, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The material breathes well. No more sweaty backs."},
    {"id": 45, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Sturdy build and doesn't wobble."},
    {"id": 46, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The adjustable armrests are a godsend."},
    {"id": 47, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Feels like a high-end chair but without the hefty price tag."},
    {"id": 48, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Rollers are smooth and don't scratch the floor."},
    {"id": 49, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The reclining feature is smooth and holds well."},
    {"id": 50, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Perfect for both office and gaming needs."},
    {"id": 51, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Height adjustment is fluid and stays in place."},
    {"id": 52, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The cushioning is just right - not too soft or too firm."},
    {"id": 53, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Customer service was helpful with inquiries."},
    {"id": 54, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Fits well in both professional and casual room settings."},
    {"id": 55, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The headrest provides adequate neck support."},
    {"id": 56, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Maintains good posture even after extended use."},
    {"id": 57, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "The lumbar support is adjustable and supportive."},
    {"id": 58, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Easy to clean and maintain."},
    {"id": 59, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "Quality far surpasses the price."},
    {"id": 60, "category": "Furniture", "product_name": "Ergonomic Chair", "review": "A great investment for anyone spending hours seated."},
]


## create sample db

In [ ]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create product reviews SQL table
table_name = "product_reviews"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("id", Integer(), primary_key=True),
    Column("category", String(16), primary_key=True),
    Column("product_name", Integer),
    Column("review", String(16), nullable=False)
)
metadata_obj.create_all(engine)

sql_database = SQLDatabase(engine, include_tables=["product_reviews"])

for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.connect() as connection:
        cursor = connection.execute(stmt)
        connection.commit()